In [5]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import Huber
from tensorflow.keras.activations import swish
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
import json

In [6]:
# ==== 1. WCZYTYWANIE DANYCH ====
# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])

# Zmieniamy nazwę kolumny i wybieramy kluczowe kolumny
glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

# Tworzymy DataFrame z JSON
meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

In [13]:
# ==== Dopasowywanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time + pd.Timedelta(hours=1)) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=2.5))]

    if not glucose_window.empty:
        glucose_max = np.nanmax(glucose_window['glucose'].values)
        glucose_min = np.nanmin(glucose_window['glucose'].values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60
        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        total_dose = row['insulin'] * 1.2

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            glucose_diff_max, glucose_diff_min, total_dose
        ])

# ✅ Print sprawdzający połączone dane
print("\n✅ Połączone dane:")
print(pd.DataFrame(merged_data, columns=['glucose', 'insulin', 'WW', 'hour', 'glucose_diff_max', 'glucose_diff_min', 'total_dose']).head(10))



✅ Połączone dane:
   glucose  insulin    WW       hour  glucose_diff_max  glucose_diff_min   
0      156      6.1  6.06   8.516667              27.0             -29.0  \
1      143      3.2  3.21  11.500000              27.0             -63.0   
2      197      1.8  1.50   9.016667             -34.0            -124.0   
3      124      5.0  5.03  23.600000              80.0               8.0   
4      152      4.0  5.75  19.800000              51.0              -3.0   
5      149      2.0  1.14   8.183333              35.0               0.0   
6      129      3.1  3.12  10.550000              55.0             -27.0   
7      112      3.0  3.00  12.833333               8.0             -33.0   
8      119      1.1  1.14  17.366667              20.0             -12.0   
9      170      4.5  5.06  20.016667              -9.0             -56.0   

   total_dose  
0        7.32  
1        3.84  
2        2.16  
3        6.00  
4        4.80  
5        2.40  
6        3.72  
7        3.60  


In [14]:
# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour',
    'glucose_diff_max', 'glucose_diff_min', 'total_dose'
])

# ==== Skalowanie danych ====
scalers = {}
for col in data.columns:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ==== Przygotowanie sekwencji dla GRU ====
sequence_length = 20
X, y = [], []
for i in range(len(data) - sequence_length):
    X.append(data.iloc[i:i + sequence_length].values)
    y.append(data['total_dose'].iloc[i + sequence_length])

X, y = np.array(X), np.array(y)

In [15]:
# ==== Definicja modelu GRU ====
model = Sequential()
model.add(GRU(128, return_sequences=True, input_shape=(X.shape[1], X.shape[2]), kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(GRU(64, kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

# ✅ Kompilacja modelu
model.compile(optimizer='adam', loss=Huber())

# ✅ Early stopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# ==== Trenowanie modelu ====
history = model.fit(X, y, epochs=100, batch_size=16, callbacks=[early_stopping])

# Zapis modelu
model.save('insulin_predictor.keras')

Epoch 1/100


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.7745
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5992 
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4243 
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3309 
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3541 
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3294 
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4477 
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3079 
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3437 
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2984 
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3415 
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3501 
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3267 
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3817 
Epoch 15/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3955 
Epoch 16/100
5/5 ━━━━━━━━━━━━━━━

In [ ]:
# ==== TESTY ====
test_input = pd.DataFrame([[150, 5, 5, 9.5, 20, -10, 0]], columns=data.columns)

# ✅ Skalowanie testów zgodnie z zakresem danych treningowych
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Ustawiamy sekwencję testową o długości 20
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input.iloc[0].values

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scalers['total_dose'].inverse_transform([[predicted_dose]])[0][0]
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

🔥 Proponowana dawka: 4.43 jednostek
